In [1]:
filename = "/home/begood/Videos/billiejean.mp4"

In [2]:
frames_dir = "./frames"

In [3]:
!rm -rf $frames_dir results
!mkdir $frames_dir
!ffmpeg -i $filename -vf fps=24 $frames_dir/out%d.png

frame= 7100 fps=995 q=-0.0 Lsize=N/A time=00:04:55.83 bitrate=N/A speed=41.5x    
video:2167013kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import argparse
import time
import cv2
import torch
import torch.nn as nn
from models import FastDVDnet
from fastdvdnet import denoise_seq_fastdvdnet
from utils import batch_psnr, init_logger_test, \
            variable_to_cv2_image, remove_dataparallel_wrapper, open_sequence, close_logger, get_imagenames

In [6]:
args={}
args['save_path'] = "results"
noise_sigma = 25 / 255
batch = 25
#args['model_spatial_file'] = "model_spatial.pth"
model = "./model.pth"
args['suffix'] = ""
save_noisy = False
save_results = True
gray = False

In [7]:
args['cuda'] = torch.cuda.is_available()

In [8]:
NUM_IN_FR_EXT = 5 # temporal size of patch
#MC_ALGO = 'DeepFlow' # motion estimation algorithm
OUTIMGEXT = '.png' # output images format

In [9]:
# If save_path does not exist, create it
if not os.path.exists(args['save_path']):
    os.makedirs(args['save_path'])

In [10]:
# Sets data type according to CPU or GPU modes
if args['cuda']:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [11]:
# Create models
model_temp = FastDVDnet(num_input_frames=NUM_IN_FR_EXT)
state_temp_dict = torch.load(model)

In [12]:
state_temp_dict = torch.load(model)
if args['cuda']:
    device_ids = [0]
    model_temp = nn.DataParallel(model_temp, device_ids=device_ids).cuda()
else:
    # CPU mode: remove the DataParallel wrapper
    state_temp_dict = remove_dataparallel_wrapper(state_temp_dict)
model_temp.load_state_dict(state_temp_dict)

In [13]:
model_temp.eval()
print("Eval mode set")

Eval mode set


In [14]:
nb_file = len(os.listdir(frames_dir))
# Get ordered list of filenames
files = get_imagenames(frames_dir)

In [15]:
def save_out_seq(seqclean, save_dir, sigmaval, start_idx):
    """Saves the denoised and noisy sequences under save_dir
    """
    seq_len = seqclean.size()[0]
    for idx in range(seq_len):
        save_img(seqclean, save_dir, sigmaval, start_idx,idx)

In [16]:
def save_img(seq, save_dir, sigmaval, start_idx, idx):
    out_name = os.path.join(save_dir,\
                ('n{}_DVDnet_{:0>15}').format(sigmaval, start_idx+idx) + OUTIMGEXT)

    outimg = variable_to_cv2_image(seq[idx].unsqueeze(dim=0))
    cv2.imwrite(out_name, outimg)

In [17]:
start_time = time.time()

for fidx in range(0, len(files), batch):
    subfiles = files[fidx:fidx+batch]
    seq, _, _ = open_sequence(subfiles, frames_dir,\
                    gray,\
                    expand_if_needed=False,\
                    start_index=0, qty=batch)
    # process data
    with torch.no_grad():
        seq = torch.from_numpy(seq).to(device)
        seq_time = time.time()

        # Add noise
        noise = torch.empty_like(seq).normal_(mean=0, std=noise_sigma).to(device)
        seqn = seq + noise
        noisestd = torch.FloatTensor([noise_sigma]).to(device)
        print("Noise applied")

        denframes = denoise_seq_fastdvdnet(seq=seqn,\
                                        noise_std=noisestd,\
                                        temp_psz=NUM_IN_FR_EXT,\
                                        model_temporal=model_temp)
    print("Saving files starting at index {}".format(fidx))
    save_out_seq(denframes, args['save_path'], int(noise_sigma*255), fidx)
print("denoise_seq_dvdnet done")
den_time = time.time()

Image 0 ./frames/out1.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 0
Image 0 ./frames/out26.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 25
Image 0 ./frames/out51.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 50
Image 0 ./frames/out76.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 75
Image 0 ./frames/out101.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 100
Image 0 ./frames/out126.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 125
Image 0 ./frames/out151.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 150
Image 0 ./frames/out176.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 175
Image 0 ./frames/out201.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 200
Image 0 ./frames/out226.png
	Loaded sequence  ./frames
Noise applied
Savin

Noise applied
Saving files starting at index 1950
Image 0 ./frames/out1976.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 1975
Image 0 ./frames/out2001.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 2000
Image 0 ./frames/out2026.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 2025
Image 0 ./frames/out2051.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 2050
Image 0 ./frames/out2076.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 2075
Image 0 ./frames/out2101.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 2100
Image 0 ./frames/out2126.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 2125
Image 0 ./frames/out2151.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 2150
Image 0 ./frames/out2176.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 2

Image 0 ./frames/out3901.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 3900
Image 0 ./frames/out3926.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 3925
Image 0 ./frames/out3951.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 3950
Image 0 ./frames/out3976.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 3975
Image 0 ./frames/out4001.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 4000
Image 0 ./frames/out4026.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 4025
Image 0 ./frames/out4051.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 4050
Image 0 ./frames/out4076.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 4075
Image 0 ./frames/out4101.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 4100
Image 0 ./frames/out4126.png
	Loaded sequence 

Saving files starting at index 5825
Image 0 ./frames/out5851.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 5850
Image 0 ./frames/out5876.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 5875
Image 0 ./frames/out5901.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 5900
Image 0 ./frames/out5926.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 5925
Image 0 ./frames/out5951.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 5950
Image 0 ./frames/out5976.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 5975
Image 0 ./frames/out6001.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 6000
Image 0 ./frames/out6026.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 6025
Image 0 ./frames/out6051.png
	Loaded sequence  ./frames
Noise applied
Saving files starting at index 6050
Image 0 ./

In [18]:
# Save outputs
#save_out_seq(denframes, args['save_path'], int(noise_sigma*255))

In [19]:
!ffmpeg -framerate 24 \
    -pattern_type glob \
    -i 'results/*.png' \
    -c:v libx264  \
    -r 30 \
    -pix_fmt yuv420p10le \
    -map_chapters 0 \
    -preset fast \
    -crf 21 \
    -c:a copy \
    out.mkv